In [ ]:
pip install --upgrade jupyter-dash dash dash-leaflet plotly pandas numpy pymongo

In [1]:
from dash import Dash, dcc, html, dash_table as dt
import dash
import dash_leaflet as dl
import plotly.express as px
from dash.dependencies import Input, Output, State

import os, base64
import numpy as np
import pandas as pd
from pymongo import MongoClient

# =======================
# DB or CSV bootstrap
# =======================
USE_DB = True
try:
    _client = MongoClient("mongodb://localhost:27017", serverSelectionTimeoutMS=2000)
    _client.admin.command("ping")
except Exception:
    USE_DB = False

if USE_DB:
    from animalShelter import AnimalShelter
    animals = AnimalShelter(host="localhost", port=27017, db="AAC", col="animals")
    df = pd.DataFrame.from_records(animals.read({}))
else:
    df = pd.read_csv("aac_shelter_outcomes.csv")

print("Rows loaded:", len(df))
print(df.columns.tolist())

# =======================
# Clean & prep dataframe
# =======================
if '' in df.columns:
    df = df.drop(columns=[''])

for col in ['location_lat', 'location_long', 'age_upon_outcome_in_weeks']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

for col in ['breed', 'name', 'animal_type', 'sex_upon_outcome', 'outcome_type']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

LAT_COL = 'location_lat'
LON_COL = 'location_long'

# Age stats for slider (compute once)
age_series = pd.to_numeric(df["age_upon_outcome_in_weeks"], errors="coerce")
age_min = int(np.nanmin(age_series.values)) if len(age_series) else 0
age_max = int(np.nanmax(age_series.values)) if len(age_series) else 300
try:
    q = np.quantile(age_series.dropna(), [0, 0.25, 0.5, 0.75, 1.0]).astype(int)
    marks = {int(v): {"label": f"{int(v)}w"} for v in q}
except Exception:
    marks = {age_min: {"label": f"{age_min}w"}, age_max: {"label": f"{age_max}w"}}

# =======================
# Dash app + theme
# =======================
external_stylesheets = []
app = Dash(__name__, external_stylesheets=external_stylesheets)

# Inject a modern, Apple-ish dark theme
app.index_string = """
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>Grazioso • Dashboard</title>
        {%favicon%}
        {%css%}
        <style>
            :root{
                --bg: #0b0f17;
                --bg-elev: #111827;
                --card: #121827;
                --card-2: #0f1624;
                --muted: #9aa4b2;
                --text: #e5e7eb;
                --accent: #3b82f6;
                --accent-2: #22d3ee;
                --ring: rgba(59,130,246,.35);
                --radius: 18px;
                --shadow: 0 10px 30px rgba(0,0,0,.35);
                --border: 1px solid rgba(255,255,255,.06);
            }
            html, body, #react-entry-point, #_dash-app-content{
                height: 100%;
            }
            body{
                margin:0;
                background:
                  radial-gradient(1200px 600px at 10% 0%, rgba(34,211,238,.08), transparent 40%),
                  radial-gradient(900px 500px at 90% -10%, rgba(59,130,246,.10), transparent 50%),
                  var(--bg);
                color: var(--text);
                font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, "Helvetica Neue", Arial, "Apple Color Emoji","Segoe UI Emoji";
            }
            .container{
                max-width: 1200px;
                margin: 24px auto 64px;
                padding: 0 16px;
            }
            .hero{
                background: linear-gradient(180deg, rgba(255,255,255,0.03), rgba(255,255,255,0.02));
                border: var(--border);
                border-radius: calc(var(--radius) + 6px);
                padding: 24px 24px;
                box-shadow: var(--shadow);
                backdrop-filter: saturate(140%) blur(6px);
            }
            .hero h1{
                font-size: 26px;
                margin: 0 0 8px 0;
            }
            .hero p{
                margin: 6px 0 0;
                color: var(--muted);
            }
            .chips{
                display:flex; gap:10px; flex-wrap:wrap; margin-top:10px;
            }
            .chip{
                padding: 6px 12px; border-radius: 999px;
                background: rgba(255,255,255,.06);
                border: var(--border);
                color: var(--text);
                font-size: 12px;
            }
            .grid{
                display:grid;
                grid-template-columns: 1fr;
                gap: 18px;
                margin-top: 18px;
            }
            @media (min-width: 980px){
                .grid-2{ grid-template-columns: 1fr 1fr; }
            }
            .card{
                background: linear-gradient(180deg, var(--card), var(--card-2));
                border: var(--border);
                border-radius: var(--radius);
                box-shadow: var(--shadow);
                overflow: hidden;
            }
            .card-header{
                display:flex; align-items:center; justify-content:space-between;
                padding: 14px 16px; border-bottom: var(--border);
            }
            .card-title{ font-weight:600; letter-spacing:.2px; }
            .card-body{ padding: 14px 16px; }

            /* Segmented radio buttons */
            .segmented{
                display:inline-flex; background: rgba(255,255,255,.06); border-radius: 14px; padding:4px;
                border: var(--border);
            }
            .segmented label{
                padding: 6px 10px; border-radius: 10px; margin: 0 2px; cursor:pointer; color: var(--muted);
            }
            .segmented input{ display:none; }
            .segmented input:checked + span{
                background: linear-gradient(180deg, rgba(59,130,246,.25), rgba(59,130,246,.15));
                color: var(--text);
                box-shadow: inset 0 0 0 1px var(--ring);
            }

            /* DataTable theme */
            .dash-table-container .dash-spreadsheet-container{
                background: transparent !important;
            }
            .dash-table-container .dash-spreadsheet-inner td, 
            .dash-table-container .dash-spreadsheet-inner th{
                border: none !important;
            }
            .dash-table-container .previous-next-container button{
                background: rgba(255,255,255,.06);
                border: var(--border);
                color: var(--text);
                border-radius: 8px;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
"""

# Encode logo once (base64) so it inlines cleanly in the layout.
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
initial_selected_rows = [0] if len(df) > 0 else []

# -----------------------
# Layout (cards everywhere)
# -----------------------
def segmented_radio(id):
    # Custom segmented control look
    return html.Div(
        className="segmented",
        children=[
            html.Label([
                dcc.RadioItems(
                    id=id,
                    options=[
                        {'label': 'Water Rescue', 'value': 'water'},
                        {'label': 'Mountain/Wilderness', 'value': 'mount'},
                        {'label': 'Disaster/Tracking', 'value': 'disaster'},
                        {'label': 'Reset', 'value': 'reset'}
                    ],
                    value='reset',
                    inputStyle={"display":"none"},
                    labelStyle={"display":"none"}  # hide natives; we render spans below
                ),
                # Visual buttons (click still toggles via label)
                dcc.Input(type="radio", name="fake", checked=True, style={"display":"none"})
            ], style={"display":"none"}),
            # Faux buttons (we’ll just rely on the native; this block is for style only)
        ]
    )

app.layout = html.Div(
    className="container",
    children=[

        # HERO
        html.Div(className="hero", children=[
            html.Div(style={"display":"flex","gap":"16px","alignItems":"center","flexWrap":"wrap"}, children=[
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={"width":"56px","height":"56px","borderRadius":"16px","border":"1px solid rgba(255,255,255,.08)"}
                ),
                html.Div(children=[
                    html.H1("Grazioso Salvare • Animal Outcomes"),
                    html.P("Modern, interactive dashboard with filters, geolocation, and real-time aggregation.")
                ]),
            ]),
            html.Div(className="chips", children=[
                html.Span("MongoDB + Dash", className="chip"),
                html.Span("JWT-ready API", className="chip"),
                html.Span("Aggregation Charts", className="chip"),
                html.Span("Leaflet Map", className="chip"),
            ])
        ]),

        # FILTERS CARD
        html.Div(className="card", children=[
            html.Div(className="card-header", children=[
                html.Span("Filters", className="card-title")
            ]),
            html.Div(className="card-body", children=[
                # First row: presets (left), outcome (right)
                html.Div(className="grid", children=[
                    html.Div(children=[
                        html.Label("Preset", style={"display":"block","marginBottom":"6px","color":"#9aa4b2"}),
                        dcc.RadioItems(
                            id='filter-type',
                            options=[
                                {'label': 'Water Rescue', 'value': 'water'},
                                {'label': 'Mountain/Wilderness', 'value': 'mount'},
                                {'label': 'Disaster/Tracking', 'value': 'disaster'},
                                {'label': 'Reset', 'value': 'reset'}
                            ],
                            value='reset',
                            labelStyle={'display': 'inline-block', 'marginRight': '12px', 'padding':'6px 10px',
                                        'borderRadius':'10px', 'background':'rgba(255,255,255,.06)',
                                        'border':'1px solid rgba(255,255,255,.06)'}
                        ),
                    ]),
                    html.Div(children=[
                        html.Label("Outcome", style={"display":"block","marginBottom":"6px","color":"#9aa4b2"}),
                        dcc.Dropdown(
                            id="outcome-filter",
                            options=([{"label": "ALL", "value": "ALL"}] +
                                     [{"label": x, "value": x}
                                      for x in sorted(df["outcome_type"].dropna().unique().tolist())]),
                            value="ALL",
                            clearable=False
                        )
                    ])
                ], style={"marginBottom":"12px"}),

                # Second row: age + breed
                html.Div(className="grid", children=[
                    html.Div(children=[
                        html.Label("Age (weeks)", style={"display":"block","marginBottom":"6px","color":"#9aa4b2"}),
                        dcc.RangeSlider(
                            id="age-filter",
                            min=age_min,
                            max=age_max,
                            step=1,
                            value=[age_min, age_max],
                            updatemode="mouseup",
                            allowCross=False,
                            persistence=True,
                            persistence_type="session",
                            marks=marks,
                            tooltip={"placement": "bottom", "always_visible": False}
                        ),
                        html.Div(id="age-filter-label", style={"fontSize":"12px", "marginTop":"6px", "color":"#9aa4b2"}),
                    ]),
                    html.Div(children=[
                        html.Label("Breed contains", style={"display":"block","marginBottom":"6px","color":"#9aa4b2"}),
                        dcc.Input(
                            id="breed-filter",
                            type="text",
                            placeholder="e.g., Retriever",
                            value="",
                            debounce=True,
                            style={"width":"100%","padding":"10px 12px","borderRadius":"12px",
                                   "background":"rgba(255,255,255,.06)","border":"1px solid rgba(255,255,255,.08)",
                                   "color":"#e5e7eb"}
                        ),
                        html.Button("Reset age", id="reset-age", n_clicks=0,
                                    style={"marginTop":"10px","padding":"8px 12px","borderRadius":"10px",
                                           "background":"rgba(59,130,246,.15)","border":"1px solid rgba(59,130,246,.35)",
                                           "color":"#e5e7eb", "cursor":"pointer"})
                    ])
                ])
            ])
        ]),

        # TABLE CARD
        html.Div(className="card", children=[
            html.Div(className="card-header", children=[
                html.Span("Animals Table", className="card-title")
            ]),
            html.Div(className="card-body", children=[
                dt.DataTable(
                    id='datatable-id',
                    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                    data=df.to_dict('records'),
                    editable=False,
                    sort_action="native",
                    sort_mode="multi",
                    column_selectable="single",
                    row_selectable="single",
                    row_deletable=False,
                    selected_columns=[],
                    selected_rows=initial_selected_rows,
                    page_action="native",
                    page_current=0,
                    page_size=10,
                    style_as_list_view=True,
                    style_header={
                        "backgroundColor":"#0f1624",
                        "color":"#e5e7eb",
                        "fontWeight":"600",
                        "border":"1px solid rgba(255,255,255,.06)"
                    },
                    style_cell={
                        "backgroundColor":"transparent",
                        "color":"#d6dae0",
                        "padding":"12px",
                        "border":"1px solid rgba(255,255,255,.03)",
                        "fontSize":"13px"
                    },
                    style_data_conditional=[
                        {
                            "if":{"state":"selected"},
                            "backgroundColor":"rgba(59,130,246,.15)",
                            "border":"1px solid rgba(59,130,246,.45)"
                        },
                        {
                            "if":{"row_index":"odd"},
                            "backgroundColor":"rgba(255,255,255,.02)"
                        },
                        {
                            "if":{"column_editable":True},
                            "color":"#fff"
                        }
                    ],
                    style_table={"borderRadius":"12px", "overflow":"hidden"}
                )
            ])
        ]),

        # ANALYTICS + MAP (two cards grid)
        html.Div(className="grid grid-2", children=[

            html.Div(className="card", children=[
                html.Div(className="card-header", children=[
                    html.Span("Top Breeds (current view)", className="card-title")
                ]),
                html.Div(className="card-body", children=[
                    html.Div(id='graph-id')
                ])
            ]),

            html.Div(className="card", children=[
                html.Div(className="card-header", children=[
                    html.Span("Location", className="card-title")
                ]),
                html.Div(className="card-body", children=[
                    html.Div(id='map-id', style={"borderRadius":"14px","overflow":"hidden"})
                ])
            ])

        ])
    ]
)

# =======================
# Callbacks
# =======================

@app.callback(
    Output("age-filter-label", "children"),
    Input("age-filter", "value")
)
def show_age_label(v):
    if not v:
        return ""
    lo, hi = map(int, v)
    to_yr = lambda w: f"{w/52:.1f}y"
    return f"{lo}–{hi} weeks  ({to_yr(lo)}–{to_yr(hi)})"

@app.callback(
    Output("age-filter", "value"),
    Input("reset-age", "n_clicks"),
    prevent_initial_call=True
)
def reset_age(_):
    return [age_min, age_max]

# Filter the DataTable (radio presets + age)
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns'),
     Output('datatable-id', 'selected_rows')],
    [Input('filter-type', 'value'),
     Input('age-filter', 'value')]
)
def update_dashboard(filter_type, age_range):
    dff = df.copy()

    if filter_type == 'water':
        mask = (
            (dff['animal_type'] == 'Dog') &
            (dff['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
            (dff['sex_upon_outcome'] == 'Intact Female') &
            (dff['age_upon_outcome_in_weeks'].between(26.0, 156.0, inclusive='both'))
        )
        dff = dff[mask]

    elif filter_type == 'mount':
        mount_breeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
        mask = (
            (dff['animal_type'] == 'Dog') &
            (dff['breed'].isin(mount_breeds)) &
            (dff['sex_upon_outcome'] == 'Intact Male') &
            (dff['age_upon_outcome_in_weeks'].between(26.0, 156.0, inclusive='both'))
        )
        dff = dff[mask]

    elif filter_type == 'disaster':
        disaster_breeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        mask = (
            (dff['animal_type'] == 'Dog') &
            (dff['breed'].isin(disaster_breeds)) &
            (dff['sex_upon_outcome'] == 'Intact Male') &
            (dff['age_upon_outcome_in_weeks'].between(20.0, 300.0, inclusive='both'))
        )
        dff = dff[mask]

    if age_range and 'age_upon_outcome_in_weeks' in dff.columns:
        lo, hi = map(int, age_range)
        dff = dff[dff['age_upon_outcome_in_weeks'].between(lo, hi, inclusive='both')]

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    data = dff.to_dict('records')
    selected_rows = [0] if len(dff) > 0 else []
    return data, columns, selected_rows

# Bar chart (Top Breeds) using Mongo agg or pandas fallback
@app.callback(
    Output('graph-id', "children"),
    [
        Input('filter-type', 'value'),
        Input('outcome-filter', 'value'),
        Input('age-filter', 'value'),
        Input('breed-filter', 'value'),
        Input('datatable-id', "derived_viewport_data"),
    ]
)
def update_graphs(filter_type, outcome_val, age_range, breed_text, view_data):
    match = {}

    if filter_type == 'water':
        match.update({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif filter_type == 'mount':
        match.update({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard","German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        })
    elif filter_type == 'disaster':
        match.update({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher","German Shepard","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        })

    if outcome_val and outcome_val != "ALL":
        match["outcome_type"] = outcome_val

    if age_range:
        lo, hi = map(int, age_range)
        match.setdefault("age_upon_outcome_in_weeks", {})
        match["age_upon_outcome_in_weeks"].update({"$gte": lo, "$lte": hi})

    if breed_text:
        match["breed"] = {"$regex": breed_text, "$options": "i"}

    pipeline = [
        {"$match": match if match else {}},
        {"$group": {"_id": {"breed": "$breed"}, "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 20},
        {"$project": {"_id": 0, "breed": "$_id.breed", "count": 1}}
    ]

    if not USE_DB:
        dff = df.copy()
        for k, v in match.items():
            if k == "breed" and isinstance(v, dict) and "$regex" in v:
                dff = dff[dff["breed"].str.contains(v["$regex"], case=False, na=False)]
            elif k == "age_upon_outcome_in_weeks" and isinstance(v, dict):
                lo2 = v.get("$gte", dff["age_upon_outcome_in_weeks"].min())
                hi2 = v.get("$lte", dff["age_upon_outcome_in_weeks"].max())
                dff = dff[dff["age_upon_outcome_in_weeks"].between(lo2, hi2, inclusive="both")]
            else:
                if isinstance(v, dict) and "$in" in v:
                    dff = dff[dff[k].isin(v["$in"])]
                else:
                    dff = dff[dff[k] == v]

        top = (
            dff.groupby("breed", dropna=False)
               .size()
               .reset_index(name="count_breed")
               .sort_values("count_breed", ascending=False)
               .head(20)
        )
    else:
        agg = list(animals.collection.aggregate(pipeline))
        if not agg:
            return [html.Div("No data available")]
        top = pd.DataFrame(agg).rename(columns={"count": "count_breed"})

    fig = px.bar(
        top, x='breed', y='count_breed',
        title="Top Breeds in Current View",
    )
    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font_color='#e5e7eb',
        margin=dict(l=0,r=0,t=40,b=0),
        xaxis=dict(showgrid=False),
        yaxis=dict(gridcolor='rgba(255,255,255,.08)'),
    )
    return [dcc.Graph(figure=fig, config={"displayModeBar": False})]

# Leaflet map (selected row)
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')]
)
def update_map(viewData, row_ids):
    if not viewData or not row_ids:
        return [html.Div("Select a row to view location")]

    dff = pd.DataFrame(viewData)

    if LAT_COL not in dff.columns or LON_COL not in dff.columns:
        return [html.Div("No lat/long columns found in data")]

    i = row_ids[-1]
    try:
        lat = float(dff.iloc[i][LAT_COL])
        lon = float(dff.iloc[i][LON_COL])
    except Exception:
        return [html.Div("Selected row has invalid coordinates")]

    if np.isnan(lat) or np.isnan(lon):
        return [html.Div("Selected row has missing coordinates")]

    return [
        dl.Map(style={'width': '100%', 'height': '460px'},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(str(dff.iloc[i].get('breed', 'Unknown breed'))),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(str(dff.iloc[i].get('name', 'Unknown')))
                ])
            ])
        ])
    ]

# =======================
# Jupyter run
# =======================
if __name__ == '__main__':
    app.run(jupyter_mode="inline", port=8090, debug=True)


Rows loaded: 20000
['', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long', 'age_upon_outcome_in_weeks']
